In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
# from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False
f_path = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name) # 한글처리

import warnings 
warnings.filterwarnings(action="ignore") #waring을 무시하도록 처리

# 챗봇에게 채팅한 데이터 가져오기

In [3]:
df = pd.read_csv('../project4/ChatBotData.csv')
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [5]:
df = df.drop(['A'], axis='columns').rename({'Q':'Message', 'label':'category'}, axis=1)
df

,Message,category
0,12시 땡!,0
1,1지망 학교 떨어졌어,0
2,3박4일 놀러가고 싶다,0
3,3박4일 정도 놀러가고 싶다,0
4,PPL 심하네,0
...,...,...
11818,훔쳐보는 것도 눈치 보임.,2
11819,훔쳐보는 것도 눈치 보임.,2
11820,흑기사 해주는 짝남.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,2


In [6]:
df['category'].replace({0: '일상', 1: '이별(부정)', 2: '사랑(긍정)'}, inplace=True)
df

,Message,category
0,12시 땡!,일상
1,1지망 학교 떨어졌어,일상
2,3박4일 놀러가고 싶다,일상
3,3박4일 정도 놀러가고 싶다,일상
4,PPL 심하네,일상
...,...,...
11818,훔쳐보는 것도 눈치 보임.,사랑(긍정)
11819,훔쳐보는 것도 눈치 보임.,사랑(긍정)
11820,흑기사 해주는 짝남.,사랑(긍정)
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,사랑(긍정)


In [7]:
# 데이터에 사용된 중복이 없는 전체 단어 갯수를 리턴
def get_vocab_size(df):
    results = set()
    df['Message'].str.lower().str.split().apply(results.update)
    return len(results)

In [8]:
vocab_size = get_vocab_size(df)

In [9]:
Messages = df['Message'].to_list()
encoded_Messages = [one_hot(Message, vocab_size) for Message in Messages]

In [10]:
# 데이터에서 가장 긴 문장의 단어 갯수를 리턴
def get_max_length(df):
    max_length = 0
    for row in df['Message']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

max_length = get_max_length(df)
print(max_length)

15


In [11]:
# 각 문장마다 제로패딩을 넣어서 가장 긴 문장과 길이를 동일하게 만듦
padded_Messages_encoding = pad_sequences(encoded_Messages, maxlen=max_length, padding='post')

In [12]:
# 문장의 벡터 확인
padded_Messages_encoding

array([[10270, 14169,     0, ...,     0,     0,     0],
       [ 6750,  1156,  6062, ...,     0,     0,     0],
       [10634,  8800,  2749, ...,     0,     0,     0],
       ...,
       [ 5098,   374,  1904, ...,     0,     0,     0],
       [ 5052,  6862,  4013, ...,     0,     0,     0],
       [ 2758,  5490,     0, ...,     0,     0,     0]])

In [13]:
# 분류 항목(레이블)을 원-핫 인코딩: 3가지로 분류
categories = df['category'].to_list()
def category_encode(category):
    if category == '일상':
        return [1,0,0]
    elif category == '이별(부정)':
        return [0,1,0]
    else:
        return [0,0,1]

encoded_category = [category_encode(category) for category in categories]

In [14]:
print(encoded_category[0])
print(encoded_category[1111])
print(encoded_category[11111])
print(encoded_category[7777])
print(encoded_category[5555])

[1, 0, 0]
[1, 0, 0]
[0, 0, 1]
[0, 1, 0]
[0, 1, 0]


In [15]:
model = Sequential()

# 문맥 벡터 생성 단계
model.add(Embedding(vocab_size, 5, input_length=max_length))
model.add(LSTM(64))

# 분류 단계
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))
#-----------------------------------------------
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

es = EarlyStopping(mode='min', verbose=2, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=2, save_best_only=True)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train = np.array(padded_Messages_encoding)
Y_train = np.array(encoded_category)

In [18]:
print('Train 진행')
history = model.fit(X_train, Y_train, epochs=7, callbacks=[es, mc], batch_size=10, validation_split=0.2)

Train 진행
Epoch 1/7
946/946 [==============================] - 4s 5ms/step - loss: 0.7695 - accuracy: 0.6439 - val_loss: 2.8659 - val_accuracy: 0.0000e+00
Epoch 2/7
946/946 [==============================] - 4s 4ms/step - loss: 0.5012 - accuracy: 0.8135 - val_loss: 2.5204 - val_accuracy: 0.0000e+00
Epoch 3/7
946/946 [==============================] - 4s 4ms/step - loss: 0.3494 - accuracy: 0.8715 - val_loss: 2.0384 - val_accuracy: 0.4461
Epoch 4/7
946/946 [==============================] - 4s 4ms/step - loss: 0.2227 - accuracy: 0.9241 - val_loss: 3.5012 - val_accuracy: 0.3218
Epoch 5/7
946/946 [==============================] - 4s 4ms/step - loss: 0.1643 - accuracy: 0.9466 - val_loss: 3.5242 - val_accuracy: 0.2778
Epoch 6/7
946/946 [==============================] - 4s 4ms/step - loss: 0.1314 - accuracy: 0.9555 - val_loss: 3.7374 - val_accuracy: 0.2951
Epoch 7/7
946/946 [==============================] - 4s 4ms/step - loss: 0.1111 - accuracy: 0.9625 - val_loss: 5.0890 - val_accuracy: 0.2

In [19]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))
# # 트레이닝 학습 결과
# score, acc = model.evaluate(X_train, Y_train, verbose=2)
# print('Train accuracy:', acc)

OSError: SavedModel file does not exist at: best_model.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
# from sklearn.metrics import accuracy_score
# # 테스트 결과
# Y_pred = model.predict(X_test)
# Y_test_class = np.argmax(Y_test,axis=1)
# Y_pred_class = np.argmax(Y_pred,axis=1)
# print('Train accuracy:', accuracy_score(Y_pred_class, Y_test_class))

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(Y_test_class,Y_pred_class))
print(confusion_matrix(Y_test_class,Y_pred_class))